# Les réseaux de neurones artificiels

Les réseaux de neurones artificiels sont issus de la recherche scientifique.
S'inspirant fortement du cerveau, ils peuvent sembler complexes au premier abord. Ce tutoriel vise à démystifier un modèle d'apprentissage qui n'est finalement pas aussi terrifiant que cela. (Si l'on passe outre la partie mathématique, qui, elle, peut effrayer).

Pour commencer, nous allons importer LE module qui nous sera utile : numpy.
Ce module surcouche énormément de méthodes de calcul, rendant leur utilisation triviale. Numpy propose un ensemble d'optimisations de calcul et fait même appel à du langage C pour accélerer encore ses traitements. En résumé, Numpy est simple et fait gagner du temps. Ce module est utilisé dans la majeure partie des autres modules touchant un minimum aux mathématiques, le rendant essentiel dans la plupart des cas. Par convention, nous lui affectons un alias : "np".

In [ ]:
from IPython.display import display, clear_output

import numpy as np
import matplotlib.pyplot as plt

Le but de ce tutoriel est assez simple. Nous cherchons à apprendre à la machine la porte logique XOR (OU EXCLUSIF). Pour rappel, la table de vérité du XOR est la suivante:

| X1 | X2 |--| Y |
|----|----|--|---|
|  0 |  0 |--| 0 |
|  0 |  1 |--| 1 |
|  1 |  0 |--| 1 |
|  1 |  1 |--| 0 |

Les variables X et y recoivent donc l'ensemble des valeurs de la table de vérité.
X représente maintenant les entrées de notre jeu d'entrainement. y représente les sorties attendues pour notre réseau de neurones.

In [ ]:
X = np.array([[0,0],[0,1],[1,0],[1,1]])

In [ ]:
y = np.array([[0],[1],[1],[0]])

Nous concevons maintenant notre réseau.
Ce réseau prendra une couche cachée d'une taille choisie (le paramètre "taille" à modifier). Plus la taille de la couche cachée est grande, plus le traitement devient long. Dans le cas général, il s'agit d'une valeur à choisir avec une relative précision. Dans ce cas précis, l'influence est faible.
Les poids sont alors initialisés de manière aléatoire.

Pour commencer, les poids placés entre la couche d'entrée (de taille 2 car chaque X est composé de 2 valeurs) et la couche cachée (d'une taille à définir), sont initialisés entre -1 et 1 (syn0).
Il en va de même pour les poids situés entre la couche cachée (toujours de taille à définir) et la couche de sortie (de taille 1 car seule une valeur est attendue) (syn1).
Un compteur d'entrainement est initialisé

In [ ]:
taille = ???
syn0 = 2*np.random.random((2,taille)) - 1
syn1 = 2*np.random.random((taille,1)) - 1
compteur = 0
accuracy = []

Vient ensuite la boucle d'entrainement (le nombre de passages dans la boucle est à définir en modifiant "boucles").


Les valeurs transitent via les poids de la couche d'entrée (X) à la couche cachée (l1), ces valeurs sont ensuite additionnées puis passent par une fonction sigmoïde.

De la même manière, les valeurs obtenues dans la couche cachée (l1) passent à la couche de sortie (l2).

L'étape de correction suit. La sortie (l2) est comparée aux valeurs attendues (y), puis passe par une dérivée de la sigmoïde. Nous obtenons l'importance de l'erreur sur la couche de sortie (l2_delta).

Les poids permettent de calculer l'importance d'un neurone précédent sur l'erreur actuelle. Cette pondération suivie d'une dérivée de la sigmoïde permet de trouver l'importance de l'erreur sur la couche cachée (l1_delta).

Nous modifions alors les deux couches de poids (syn0 et syn1) en fonction des erreurs trouvées.


Le résultat est alors imprimé. D'abord le nombre d'itérations d'entrainement (la somme des boucles exécutées), puis le résultat du réseau "entrainé". La progression de l'entrainement est affichée également.

In [ ]:
boucles = ???
for j in range(boucles):
    l1 = 1/(1+np.exp(-(np.dot(X,syn0))))
    l2 = 1/(1+np.exp(-(np.dot(l1,syn1))))
    l2_delta = (y - l2)*(l2*(1-l2))
    l1_delta = l2_delta.dot(syn1.T) * (l1 * (1-l1))
    syn1 += l1.T.dot(l2_delta)
    syn0 += X.T.dot(l1_delta)
    compteur += 1
    accuracy.append(1 - np.mean([abs(x) for x in l2-y])) 

print(compteur)
print(l2)
plot = plt.figure()
plt.plot(accuracy)

# La récurrence

Les réseaux de neurones tels que présentés avant se basent sur une une donnée unique afin d'y apporter une réponse instantannée.
Cependant, l'information doit souvent être traitée en prenant connaissance d'un contexte temporel. C'est notamment le cas de l'apprentissage sur du texte.

Avant toute chose, les imports, comme précédemment. Il s'agit ici des modèles d'abstraction de Keras, très utiles plus simplifier la conception de réseaux de neurones.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN, TimeDistributed
from keras.optimizers import RMSprop
import sys

Pour illustrer cela, nous allons apprendre à une machine comment écrire.

Avant toute chose, nous devons charger le texte sur lequel la machine doit apprendre.

In [ ]:
with open('data/descartes.txt', 'r') as file:
    data = file.read()

Contrairement aux valeurs numériques que le réseau a pu comprendre facilement dans la première partie, il est nécessaire de transformer notre modèle de données. Nous allons donc changer chaque lettre en un ensemble de 0 et de 1, le 1 de l'ensemble indiquant la position de la lettre dans l'alphabet.

Par la même occasion, nous mettons en place nos jeux d'entrainement et de validation. Le code peut sembler complexe au premier abord, mais il consiste juste à décaler le texte initial de 1 caractère vers la droite.
Le jeu d'entrainement est donc le texte, tel quel, tandis que le jeu de validation est, pour tout caractère du texte, le caractère suivant. On cherche donc à deviner la prochaine lettre, sachant la lettre actuelle. Pour simplifier l'entrainement, on limite l'apprentissage à des chaines de taille SEQ_LENGTH.

In [ ]:
chars = list(set(data))
VOCAB_SIZE = len(chars)
SEQ_LENGTH = ???
ix_to_char = {ix:char for ix, char in enumerate(chars)}
char_to_ix = {char:ix for ix, char in enumerate(chars)}

X = np.zeros((len(data)//SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((len(data)//SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
for i in range(0, len(data)//SEQ_LENGTH):
    X_sequence = data[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = data[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

Comme dans la premièer partie, nous concevons notre réseau de neurones, à ceci près que la tâche est grandement simplifiée par Keras. Nous avons un modèle (Sequential), constitué de LAYER_DIM couches successives, chacune de taille HIDDEN_DIM.

In [ ]:
HIDDEN_DIM = ???
LAYER_NUM = ???
boucles = 0

model = Sequential()
model.add(SimpleRNN(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(SimpleRNN(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

Afin de pouvoir comprendre le retour de la machine (un ensemble de nombres), nous définissons une méthode de traduction.

In [ ]:
def generate_text(model, length):
    ix = [np.random.randint(VOCAB_SIZE)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, VOCAB_SIZE))
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

Vient alors l'entrainement du modèle.
Pour ne pas à avoir à relancer ce code, il boucle indéfiniment. Il affiche, toutes les STEPS boucles d'entrainement, la génération d'une chaine de caractère de longueur GENERATE_LENGTH.

In [ ]:
GENERATE_LENGTH = 100
STEPS = 3
BATCH_SIZE = 32

print(generate_text(model, GENERATE_LENGTH))
while True:
    boucles += STEPS
    nb_epoch = 0
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=0, epochs=STEPS)
    nb_epoch += STEPS
    puts = generate_text(model, GENERATE_LENGTH)

    clear_output()
    print(boucles)
    print(puts)

Voilà qui conclut la partie sur les réseaux de neurones. Il ne s'agit là que d'une présentation très basique qui reste limitée en comparaison des possibilités qu'offrent ces modèles.